# Complete Code, it helped me to get score 0.8897 on leaderboard

# **Data Reading**

In [ ]:
from tqdm.autonotebook import tqdm
import numpy as np # linear algebra
import pandas as pd

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
cat_test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')

# Data encoding

In [ ]:
cols = [i for i in train.columns if train[i].dtypes=='object']
cols

In [ ]:
for i in cols:
    print(train[i].unique())
    
# we will lable them manualy like a =1 , b =2 so ab = 26+(1*2) = 28,  why we add so that they are above 26 as z ==26  so like that we will do it for all, yes it is to be done manually

In [ ]:
labels = {}
import string
test_list = list(string.ascii_uppercase) 
print(test_list)
for i,j in zip(range(1,27),test_list):
    labels[j] = i
    

In [ ]:
for p,q in zip(range(1,27),test_list):
    for i,j in zip(range(1,27),test_list):
        labels[q+j] = 26+(p*i)

# we are realy with out lables , lets change our value

In [ ]:
for i in tqdm(cols):
    for p,q in labels.items():
        train[i] = train[i].replace(p,q)

In [ ]:
for i in tqdm(cols):
    for p,q in labels.items():
        test[i] = test[i].replace(p,q)

In [ ]:
train = train.drop(columns='id')
test = test.drop(columns='id')

In [ ]:
train.groupby('target').nunique()

# Data Balancing

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

X = train.drop(columns='target')
Y = train['target']

# special case
over = RandomOverSampler(sampling_strategy=0.5)
under = RandomOverSampler(sampling_strategy=0.8)

x1, y1 = over.fit_resample(X,Y)
c_x, c_y = under.fit_resample(x1, y1)
print(len(c_y))
# # we will create four models 

# Hypermeter Optimization

Special thanks to [https://www.kaggle.com/haichaoshang/tps-mar2021-lgbm-optuna](http://)

In [ ]:
import optuna.integration.lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
import lightgbm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import xgboost as xgb

# **Here i have run it only for one trial, but actually for better score, i run it for 30 trials, you can tune it**

In [ ]:
def objective(trial, data = X, target = Y):
    X_train, X_val, y_train, y_val = train_test_split(X, Y,stratify=Y, test_size = 0.2, random_state = 0)

    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 50, 3000),
        'max_bin': trial.suggest_int('max_bin', 200, 400),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log = True),
        'subsample': trial.suggest_float('subsample', 0.1, 0.8),
        'random_seed': 42,
        'task_type': 'GPU',
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'bootstrap_type': 'Poisson'
    }
    
    model = CatBoostClassifier(**params)  
    model.fit(X_train, y_train, eval_set = [(X_val,y_val)], early_stopping_rounds = 222, verbose = False)
    y_pred = model.predict_proba(X_val)[:,1]
    roc_auc = roc_auc_score(y_val, y_pred)

    return roc_auc

In [ ]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 1)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)

In [ ]:
def objective(trial):
    X_train, X_val, y_train, y_val = train_test_split(X, Y,stratify=Y, test_size = 0.2, random_state = 0)
    params = {
        "verbosity": 0,
        "objective": "binary:logistic",
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        "eval_metric": "auc",
        "eta" : trial.suggest_float("eta", 1e-8, 1.0, log=True),
        "booster": trial.suggest_categorical("booster", ["dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        'tree_method' : 'gpu_hist',
        'eval_metric': 'auc',
        'random_seed': 42,
    }

    model = XGBClassifier(**params) 
    model.fit(X_train, y_train, eval_set = [(X_val,y_val)], early_stopping_rounds = 222, verbose = False)
    y_pred = model.predict_proba(X_val)[:,1]
    roc_auc = roc_auc_score(y_val, y_pred)
    return roc_auc

In [ ]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 1)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)

In [ ]:
# optuna.visualization.plot_param_importances(study)

In [ ]:
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
def scaling(X_train, X_test, trial):
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    test_data = sc.transform(trial)
    return X_train, X_test, test_data


def lgbm_model(X_train, X_test, Y_train,Y_test,test_data):
    
    params =  {'reg_alpha': 9.883045220708368,
         'reg_lambda': 1.8406970907815618e-05, 
         'num_leaves': 105, 'learning_rate': 0.0363323342120501,
         'max_depth': 9, 'n_estimators': 7645, 'min_child_samples': 30, 
         'min_child_weight': 0.6126344551951962, 'subsample': 0.305413058674825,
         'colsample_bytree': 0.9557961982690266, 'random_state': 2021,
        'metric' : 'auc',
        'device_type' : 'gpu'}
    
    model = lightgbm.LGBMClassifier(**params)
    model.fit(X_train , Y_train , eval_set = [(X_test , Y_test)] , early_stopping_rounds = 200 , \
             verbose = False)
    preds = model.predict_proba(X_test,num_iteration=model.best_iteration_)[:,1]
    test_preds = model.predict_proba(test_data,num_iteration=model.best_iteration_)[:,1]
    return preds,test_preds




def xgb_model(X_train, X_test, Y_train,Y_test,test_data):
    
    params =  {'max_depth': 8, 'eta': 0.1917070083625873, 'booster': 'dart', 
               'lambda': 0.003158888201683581, 'alpha': 0.16427957283436528,
            'tree_method' : 'gpu_hist',
        'eval_metric': 'auc',
        'random_seed': 42 }
    
    model = XGBClassifier(**params)  
    model.fit(X_train, Y_train, eval_set = [(X_test,Y_test)], early_stopping_rounds = 222, verbose = False)
    preds = model.predict_proba(X_test)[:,1]
    test_preds = model.predict_proba(test_data)[:,1]
    return preds,test_preds

**Again runing for only 2 splits**

In [ ]:
score = []
final = []
kfold = StratifiedKFold(n_splits=2, random_state=24, shuffle=True)

for Train, Test in kfold.split(X,Y):
    X_train, X_test = X.iloc[Train], X.iloc[Test]
    Y_train, Y_test = Y.iloc[Train], Y.iloc[Test]
#     X_train, X_test, test_data = scaling(X_train, X_test,test)
    #====== MODEL =======================
    params = {'max_depth': 6, 'learning_rate': 0.05, 'n_estimators': 2997, 
              'max_bin': 328, 'min_data_in_leaf': 28,
              'l2_leaf_reg': 0.003018532125919831, 
              'subsample': 0.6347767905576908,
             'random_seed': 42,
        'task_type': 'GPU',
             'eval_metric': 'AUC',
             'bootstrap_type': 'Poisson'}
    
    model = CatBoostClassifier(**params)  
    model.fit(X_train, Y_train, eval_set = [(X_test,Y_test)], early_stopping_rounds = 222, verbose = False)
    preds = model.predict_proba(X_test)[:,1]
    test_preds = model.predict_proba(test)[:,1]
    
    #====================================
    roc = roc_auc_score(Y_test, preds)
    print(roc)
    score.append(roc)
    final.append(test_preds)
average_score = np.mean(score)
print('The average roc is ', average_score)
    


In [ ]:
score = []
u_final = []
kfold = StratifiedKFold(n_splits=2, random_state=24, shuffle=True)

for Train, Test in kfold.split(X , Y):
    X_train, X_test = X.iloc[Train], X.iloc[Test]
    Y_train, Y_test = Y.iloc[Train], Y.iloc[Test]
#     X_train, X_test, test_data = scaling(X_train, X_test,test)
    #====== MODEL =======================
    preds,test_preds = lgbm_model(X_train, X_test, Y_train, Y_test,test)
    #====================================
    roc = roc_auc_score(Y_test, preds)
    print(roc)
    score.append(roc)
    u_final.append(test_preds)
average_score = np.mean(score)
print('The average roc is ', average_score)
    


In [ ]:
score = []
o_final = []
kfold = StratifiedKFold(n_splits=2, random_state=24, shuffle=True)

for Train, Test in kfold.split(X , Y):
    X_train, X_test = X.iloc[Train], X.iloc[Test]
    Y_train, Y_test = Y.iloc[Train], Y.iloc[Test]
#     X_train, X_test, test_data = scaling(X_train, X_test,test)
    #====== MODEL =======================
    preds,test_preds = xgb_model(X_train, X_test, Y_train, Y_test,test)
    #====================================
    roc = roc_auc_score(Y_test, preds)
    print(roc)
    score.append(roc)
    o_final.append(test_preds)
average_score = np.mean(score)
print('The average roc is ', average_score)
    


In [ ]:
r1= np.mean(final,0)  # cat
r2 = np.mean(o_final,0) #xgb
r3 = np.mean(u_final,0)  # lgbm

In [ ]:
r_9 = r1*0.2+r2*0.2+r3*0.6
r_9

In [ ]:
r_8 = r1*0.3+r2*0.1+r3*0.6
r_8

In [ ]:
r = r_9*0.4+r_8*0.6
r

In [ ]:
sub['target'] = r

In [ ]:
sub.to_csv('sub.csv',index=False)